# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started. Once you have completed the code you can download the notebook for making a submission.

In [1]:
import numpy as np
import os
from imageio import imread
from skimage.transform import resize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from tensorflow import keras
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

**data path: /home/datasets/Project_data**

In [124]:
# from google.colab import drive
# drive.mount('/content/drive')

In [125]:
# %cd /content/drive/My\ Drive/

In [3]:
train_doc = np.random.permutation(open('datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('datasets/Project_data/val.csv').readlines())
batch_size = 32 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

img_idx = [0,2,5,8,11,14,17,20,23,26,29]
batch_data = np.zeros((batch_size,len(img_idx),180,180,3))
batch_data.shape

In [4]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    #img_idx = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29] #create a list of image numbers you want to use for a particular video
    img_idx = np.arange(0, 30).tolist()
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size# calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,len(img_idx),100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image = resize(image,(100,100))
                    image = image/255.0
                    
                    batch_data[folder,idx,:,:,0] = (image[:,:,0])#normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image[:,:,1])#normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image[:,:,2])#normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            rem_batch_data = np.zeros((batch_size,len(img_idx),100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            rem_batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image = resize(image, (100,100))
                    image = image/255.0
                    
                    rem_batch_data[folder,idx,:,:,0] = (image[:,:,0])#normalise and feed in the image
                    rem_batch_data[folder,idx,:,:,1] = (image[:,:,1])#normalise and feed in the image
                    rem_batch_data[folder,idx,:,:,2] = (image[:,:,2])#normalise and feed in the image
                    
                rem_batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield rem_batch_data, rem_batch_labels


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [5]:
curr_dt_time = datetime.datetime.now()
train_path = 'datasets/Project_data/train'
val_path = 'datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20# choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, Flatten, BatchNormalization, Activation, Conv3D, MaxPooling3D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers
from tensorflow.keras import layers

#write your model here
model = Sequential([
    layers.Conv3D(16, 3, input_shape=(30,100,100,3),padding='same', activation='relu'),
    layers.MaxPooling3D(),
    layers.Conv3D(32, 3, padding='same', activation='relu'),
    layers.MaxPooling3D(),
    layers.Conv3D(64, 3, padding='same', activation='relu'),
    layers.MaxPooling3D(),
    layers.Flatten(),
    layers.Dense(1000, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(5, activation='relu'),
    layers.Activation('softmax')
])  

In [27]:
def get_model(width, height, depth):
  """
  Build a 3D convolutional neural network
  """
  inputs = tf.keras.Input((width, height, depth, 3))

  x = layers.Conv3D(filters=16, kernel_size=3, activation="relu")(inputs)
  x = layers.MaxPool3D(pool_size=2, padding='same')(x)
  x = layers.BatchNormalization()(x)

  x = layers.Conv3D(filters=32, kernel_size=3, activation="relu")(x)
  x = layers.MaxPool3D(pool_size=2)(x)
  x = layers.BatchNormalization()(x)

  x = layers.GlobalAveragePooling3D()(x)
  x = layers.Dense(units=512, activation="relu")(x)

  x = layers.Dropout(0.3)(x)

  outputs = layers.Dense(units=5, activation='softmax')(x)

  model= keras.Model(inputs, outputs, name="3DCNN")
  return model

In [28]:
#Build the model
model = get_model(width=100, height=100, depth=30)
model.summary()

Model: "3DCNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 100, 100, 30, 3)] 0         
_________________________________________________________________
conv3d_21 (Conv3D)           (None, 98, 98, 28, 16)    1312      
_________________________________________________________________
max_pooling3d_21 (MaxPooling (None, 49, 49, 14, 16)    0         
_________________________________________________________________
batch_normalization_18 (Batc (None, 49, 49, 14, 16)    64        
_________________________________________________________________
conv3d_22 (Conv3D)           (None, 47, 47, 12, 32)    13856     
_________________________________________________________________
max_pooling3d_22 (MaxPooling (None, 23, 23, 6, 32)     0         
_________________________________________________________________
batch_normalization_19 (Batc (None, 23, 23, 6, 32)     128   

In [29]:
optimiser = optimizers.Adam(learning_rate=0.01) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [30]:
print(model.summary())

Model: "3DCNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 100, 100, 30, 3)] 0         
_________________________________________________________________
conv3d_21 (Conv3D)           (None, 98, 98, 28, 16)    1312      
_________________________________________________________________
max_pooling3d_21 (MaxPooling (None, 49, 49, 14, 16)    0         
_________________________________________________________________
batch_normalization_18 (Batc (None, 49, 49, 14, 16)    64        
_________________________________________________________________
conv3d_22 (Conv3D)           (None, 47, 47, 12, 32)    13856     
_________________________________________________________________
max_pooling3d_22 (MaxPooling (None, 23, 23, 6, 32)     0         
_________________________________________________________________
batch_normalization_19 (Batc (None, 23, 23, 6, 32)     128   

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [15]:
optimiser = optimizers.Adam(learning_rate=0.01) #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_4 (Conv3D)            (None, 30, 100, 100, 16)  1312      
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 15, 50, 50, 16)    0         
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 15, 50, 50, 32)    13856     
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 7, 25, 25, 32)     0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 7, 25, 25, 64)     55360     
_________________________________________________________________
max_pooling3d_6 (MaxPooling3 (None, 3, 12, 12, 64)     0         
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [31]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [32]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath=filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', save_freq='epoch')

LR = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=3,mode='auto', verbose=1)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit` method to decide the number of next() calls it need to make.

In [33]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [ ]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  datasets/Project_data/train ; batch size = 32
Epoch 1/20
21/21 [==============================] - ETA: 0s - loss: 1.2325 - categorical_accuracy: 0.4615Source path =  datasets/Project_data/val ; batch size = 32
Batch:  4 Index: 32

Epoch 00001: saving model to model_init_2021-06-2602_22_50.878653/model-00001-1.23250-0.46154-29.85725-0.22000.h5
21/21 [==============================] - 134s 6s/step - loss: 1.2325 - categorical_accuracy: 0.4615 - val_loss: 29.8573 - val_categorical_accuracy: 0.2200
Epoch 2/20
21/21 [==============================] - ETA: 0s - loss: 1.0261 - categorical_accuracy: 0.5528
Epoch 00002: saving model to model_init_2021-06-2602_22_50.878653/model-00002-1.02609-0.55280-13.77014-0.25000.h5
21/21 [==============================] - 88s 4s/step - loss: 1.0261 - categorical_accuracy: 0.5528 - val_loss: 13.7701 - val_categorical_accuracy: 0.2500
Epoch 3/20
21/21 [==============================] - ETA: 0s - loss: 1.0544 - categorical_accuracy: 0.5644
Epoch